# Using Decision Tree and Random Forest for classification

In this part, we will set Decision Tree as my based algorithm, and choose Random Forest as its extension. 

We choose digits data in sklearn as the dataset which we used in our assignments to compare the performance between based Decision Tree and Random Forest.

## 1. Build a Decision Tree with sklearn

### Load digits data

We used digits data from sklearn.datasets.

In [1]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np

digits = load_digits()
X = digits['data']
y = digits['target']

### Preprocess data set

We splited the features into 3 periods, in order to satisfy the discrete requirement of Decesion Tree. Thanks prof. Julian Togelius suggested this idea for me.

In [2]:
def preprocess(data, min_d, max_d, bin_size=3):
    
    norm_data = np.clip((data - min_d) / (max_d - min_d), 0, 1)
    categorical_data = np.floor(bin_size*norm_data).astype(int)
    return categorical_data


X = preprocess(X, X.min(), X.max(), 3)

Then we used sklearn to split data into train data and test data.

In [3]:
# We will use sklearn's method for seperating the data
# This part of code is based on assignment 3
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Looking at the train/test split
print("The number of training examples: ", X_train.shape[0])
print("The number of test exampels: ", X_test.shape[0])

The number of training examples:  1347
The number of test exampels:  450


### Train Decision Tree

We use sklearn.tree.DecisionTreeClassifier to build our basic algorithm.

In [4]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(criterion='entropy', random_state=0)
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=0, splitter='best')

### Get the accuracy on test dataset

In [5]:
from sklearn.metrics import accuracy_score

y_pre = clf.predict(X_test)
acc = accuracy_score(y_test, y_pre)

print("Accuracy on the test data(Decision Tree): {0:.5}%".format(acc*100))

Accuracy on the test data(Decision Tree): 85.111%


## 2. Try Random Forest based on sklearn

Then we built a Random Forest based on sklearn, to check whether there would be an improvement of accuracy

In [6]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=50, max_features=int(1/3*len(X_train[0])))
clf.fit(X_train, y_train)

# After training, test on the same data
y_pre = clf.predict(X_test)
acc = accuracy_score(y_test, y_pre)

print("Accuracy on the test data(Random Forest): {0:.5}%".format(acc*100))

Accuracy on the test data(Random Forest): 96.222%


## 3. Implement Random Forest ourselves

### Implement Decision Tree

Because Random Forest is based on Decision Tree, we need to implement Decision Tree firstly.

In [7]:
import collections

class decisionTree:
    def __init__(self, depth, features_idx):
        self.decision_tree = None
        self.depth = depth
        self.features_idx = features_idx

    def calculateEntropy(self, y):
        class_count = collections.defaultdict(int)
        for yi in y:
            class_count[yi] += 1
        entropy = 0.0
        for key, _ in class_count.items():
            prob = float(class_count[key]) / len(y)
            entropy += prob * prob

        return 1 - entropy

    def splitData(self, X, y, att, f):
        sub_X = []
        sub_y = []
        for i, feature in enumerate(X):
            if feature[att] == f:
                sub_f = np.concatenate((feature[:att], feature[att + 1:]), axis=0)
                sub_X.append(sub_f)
                sub_y.append(y[i])
        return np.array(sub_X), np.array(sub_y) 

    def buildTree(self, X, att, y, depth):
        if np.sum(y==y[0]) == y.size:
            return y[0]
        if len(X[0]) == 0 or np.equal(X[0], X).all() or depth == self.depth:
            return max(y, key=list(y).count)

        features_n = len(X[0])
        best_info = 1000
        best_feature = -1

        # find best feature
        for i in range(features_n):
            uni_f = set(X[:,i])
            tmp_entropy = 0.0
            for f in uni_f:
                sub_X, sub_y = self.splitData(X, y, i, f)
                tmp_entropy += len(sub_X) / float(len(X)) * self.calculateEntropy(sub_y)

            if tmp_entropy < best_info:
                best_info = tmp_entropy
                best_feature = i

        best_att = att[best_feature]
        tree = {best_att: {}}

        del (att[best_feature])

        uni_f = set(X[:,best_feature])

        # based on best feature to split data and go to next recursion
        for f in uni_f:
            sub_att = att[:]
            sub_X, sub_y = self.splitData(X, y, best_feature, f)
            tree[best_att][f] = self.buildTree(sub_X, sub_att, sub_y, depth+1)
        return tree

    def fit(self, X, y):
        categorical_data = X
        att = [i for i in range(len(categorical_data[0]))]
        self.decision_tree = self.buildTree(categorical_data, att, y, 0)

    def traverse(self, tree, xi, att):
        if "{" not in str(tree):
            return tree
        root = list(tree.keys())[0]
        node = tree[root]
        label = 0
        for key in node.keys():
            if xi[root] == key:
                if type(node[key]).__name__ != 'dict':
                    label = node[key]
                else:
                    label = self.traverse(node[key], xi, att)
        return label

    def predict(self, X):
        X_test = np.array([X[:,i] for i in self.features_idx])
        X_test = X_test.T
        att = [i for i in range(len(X_test[0]))]
        ans = []
        for xi in X_test:
            label = self.traverse(self.decision_tree, xi, att)
            ans.append(label)

        return np.array(ans)

### Implement Random Forest

Firstly, we trained our Random Forest

In [8]:
import random

forest = []
n_estimators = 100
max_depth = 50

for _ in range(n_estimators):
    random_idx = random.sample([i for i in range(len(X_train))], int(2/3*len(X_train)))
    sub_X_train = np.array([X_train[i] for i in random_idx])
    sub_y_train = np.array([y_train[i] for i in random_idx])
    
    random_f_idx = random.sample([i for i in range(len(sub_X_train[0]))], int(1/3*(len(sub_X_train[0]))))
    random_f_idx = sorted(random_f_idx)
    sub_X_train = np.array([sub_X_train[:,i] for i in random_f_idx])
    sub_X_train = sub_X_train.T
     
    tree = decisionTree(max_depth, random_f_idx)
    tree.fit(sub_X_train, sub_y_train)
    forest.append(tree)

### Check predictions on test data

In [9]:
predicts = []
for tree in forest:
    y_pre = tree.predict(X_test)
    predicts.append(y_pre)

In [10]:
y_pre = []
for j in range(len(predicts[0])):
    all_y_pre = []
    for i in range(n_estimators):
        all_y_pre.append(predicts[i][j])
    y_pre.append(max(all_y_pre, key=list(all_y_pre).count))

acc = accuracy_score(y_test, y_pre)
print("Accuracy on the test data(Random Forest, own implement): {0:.5}%".format(acc*100))

Accuracy on the test data(Random Forest, own implement): 93.333%


## 4. Try different dataset -- Heart Disease UCI

We also tried Heart Disease UCI to check whether there would be some improvement. The dataset is from https://www.kaggle.com/ronitf/heart-disease-uci

### Preprocessing dataset

In [11]:
import pandas as pd

df = pd.read_csv('heart-disease-uci.csv')

data = df.values
X = data[:, :-1]
y = data[:,-1]

In [12]:
# We also need to split the data
X = preprocess(X, X.min(), X.max(), 4)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Looking at the train/test split
print("The number of training examples: ", X_train.shape[0])
print("The number of test exampels: ", X_test.shape[0])

The number of training examples:  227
The number of test exampels:  76


### Try on Decision Tree

In [16]:
clf = DecisionTreeClassifier(criterion='entropy', random_state=0)
clf.fit(X_train, y_train)

# After training, test on the same data
y_pre = clf.predict(X_test)
acc = accuracy_score(y_test, y_pre)

print("Accuracy on the test data(Decision Tree): {0:.5}%".format(acc*100))

Accuracy on the test data(Decision Tree): 64.474%


### Try on Random Forest

We tried Random Forest based on both sklearn version and our own implements

In [14]:
clf = RandomForestClassifier(n_estimators=100, max_depth=50, max_features=int(1/3*len(X_train[0])))
clf.fit(X_train, y_train)

# After training, test on the same data
y_pre = clf.predict(X_test)
acc = accuracy_score(y_test, y_pre)

print("Accuracy on the test data(Random Forest): {0:.5}%".format(acc*100))

Accuracy on the test data(Random Forest): 68.421%


In [15]:
forest = []
n_estimators = 100
max_depth = 50

for _ in range(n_estimators):
    random_idx = random.sample([i for i in range(len(X_train))], int(2/3*len(X_train)))
    sub_X_train = np.array([X_train[i] for i in random_idx])
    sub_y_train = np.array([y_train[i] for i in random_idx])
    
    random_f_idx = random.sample([i for i in range(len(sub_X_train[0]))], int(1/3*(len(sub_X_train[0]))))
    random_f_idx = sorted(random_f_idx)
    sub_X_train = np.array([sub_X_train[:,i] for i in random_f_idx])
    sub_X_train = sub_X_train.T
     
    tree = decisionTree(max_depth, random_f_idx)
    tree.fit(sub_X_train, sub_y_train)
    forest.append(tree)
    
    
predicts = []
for tree in forest:
    y_pre = tree.predict(X_test)
    predicts.append(y_pre)
    
    
y_pre = []
for j in range(len(predicts[0])):
    all_y_pre = []
    for i in range(n_estimators):
        all_y_pre.append(predicts[i][j])
    y_pre.append(max(all_y_pre, key=list(all_y_pre).count))

acc = accuracy_score(y_test, y_pre)
print("Accuracy on the test data(Random Forest, own implement): {0:.5}%".format(acc*100))

Accuracy on the test data(Random Forest, own implement): 61.842%
